## Creando inputs para el modelo AMPL a partir de archivo csv: Archivo .dat

En este cuadernillo se desarrolla el código para insertar los distintos parámetros al modelo. En vista de su practicidad, se decidió insertar los datos usando objetos de pandas creados con un archivo csv

In [1]:
# Setting out amplpy library
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["coin", "highs", "gokestrel"], # modules to install
    license_uuid="dd2ebe38-2a32-444c-9502-e6ad048779b0", # license to use
    g=globals()) # instantiate AMPL object and register magics

# Creating the model
ampl = AMPL()
ampl.eval(r"""
    # Setting groups 
    set Origin;#Conjunto de nodos generadores de carga
    set Destination;#Conjunto de nodos atractores de carga
    set CTL;#Conjunto de potenciales terminales de consolidación (hub)
    set PCTL;#Conjunto de potenciales terminales de distribución (hub)
    set Port;#Conjunto de puertos

    # Setting parameters
    param q;#Número de terminales abiertas
    
    param Fk{k in CTL};#Costo fijo de instalación de una terminal de consolidación en una terminal k ∈ K
    param Fm{m in PCTL};#Costo fijo de instalación de una terminal de distribución en una terminal m ∈ M

    param CAPk{k in CTL};#Capacidad de la terminal de consolidación k ∈ K
    param CAPm{m in PCTL};#Capacidad de la terminal de consolidación m ∈ M

    #param T_T{m in PCTL};#Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío directo
    param Cijp{i in Origin, j in Destination, p in PCTL};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío directo carretero al puerto p
    param Cijkp{i in Origin, j in Destination, k in CTL, p in Port};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal k ∈ K al puerto p
    param Cijmp{i in Origin, j in Destination, m in PCTL, p in Port};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal m ∈ M al puerto p    
    param Cijkmp{i in Origin, j in Destination,  k in CTL, m in PCTL, p in Port};#Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío a través del par de terminales k ∈ K y m ∈ M al puerto p

    param Eijp{i in Origin, j in Destination, p in PCTL};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío directo carretero al puerto p
    param Eijkp{i in Origin, j in Destination, k in CTL, p in Port};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal k ∈ K al puerto p
    param Eijmp{i in Origin, j in Destination, m in PCTL, p in Port};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal m ∈ M al puerto p    
    param Eijkmp{i in Origin, j in Destination,  k in CTL, m in PCTL, p in Port};#Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío a través del par de terminales k ∈ K y m ∈ M al puerto p
    
    param Wij{i in Origin, j in Destination};#Demanda de carga a enviar del nodo de origen i ∈ I al destino j ∈ J

    
    # Setting variables
    var Yk{k in CTL} binary;  # = 1 if CTL built at k
    var Ym{m in PCTL} binary;  # = 1 if CTL built at m
    var Ykm{k in CTL, m in PCTL} binary;  # = 1 if Yk*Ym=1
    
    var Pijp{i in Origin, j in Destination, p in PCTL};#Proporción de flujo de mercancía que se envía directamente del origen i ∈ I al destino j ∈ J por el puerto p
    var Pijkp{i in Origin, j in Destination, k in CTL, p in Port};#Proporción de flujo que se envía del origen i ∈ I al destino j ∈ J a través de la terminal k ∈ K al puerto p
    var Pijmp{i in Origin, j in Destination, m in PCTL, p in Port};#Proporción de flujo que se envía del origen i ∈ I al destino j ∈ J a través de la terminal m ∈ M al puerto p
    var Pijkmp{i in Origin, j in Destination,  k in CTL, m in PCTL, p in Port};#Proporción de flujo que se envía del origen i ∈ I al destino j ∈ J a través de las terminales k ∈ K y m ∈ M al puerto p
    
    
    # Setting Objective Function
    
    minimize total_cost:
        sum {k in CTL} Yk[k]*Fk[k] + sum {m in PCTL} Ym[m]*Fm[m]
        + sum {i in Origin, j in Destination, p in PCTL} Pijp[i,j,p]*Wij[i,j]*(Cijp[i,j,p]+Eijp[i,j,p])
        + sum {i in Origin, j in Destination, k in PCTL, p in Port} Pijkp[i,j,k,p]*Wij[i,j]*(Cijkp[i,j,k,p]+Cijkp[i,j,k,p])
        + sum {i in Origin, j in Destination, m in CTL, p in Port} Pijmp[i,j,m,p]*Wij[i,j]*(Cijmp[i,j,m,p]+Cijmp[i,j,m,p])
        + sum {i in Origin, j in Destination, k in CTL, m in PCTL, p in Port} Pijkmp[i,j,k,m,p]*Wij[i,j]*(Cijkmp[i,j,k,m,p]+Eijkmp[i,j,k,m,p])

       ;

    # Restrictions
    subject to chosen_CTL:#que q terminales serán seleccionadas
        (sum{k in CTL} Yk[k]) + (sum{m in PCTL} Yk[m]) = q;
        
    subject to capacity {i in Origin, j in Destination}:#garantiza que el flujo de carga entre los nodos i y j es ruteado a través del envío directo o a través de las terminales que se encuentren disponibles
        (sum{p in Port} Pijp[i,j,p]) + (sum{k in CTL,p in Port} Pijkp[i,j,k,p]) + (sum{m in PCTL,p in Port} Pijkp[i,j,m,p]) 
        + (sum{k in CTL, m in PCTL, p in Port} Pijkmp[i,j,k,m,p]) = 1;
        
    subject to binary1 {k in CTL, m in PCTL}:#garantizan la resolución del problema de multiplicación de las variables binarias.
        Yk[k] + Ym[m] <= 1 + Ykm[k,m];
        
    subject to binary2 {k in CTL, m in PCTL}:#garantizan la resolución del problema de multiplicación de las variables binarias.
        2*Ykm[k,m] <= Yk[k] + Ym[m];
    
    subject to flow1 {k in CTL}:#garantizan que el flujo de carga que maneja cada terminal es menor o igual a la capacidad instalada
        (sum{i in Origin, j in Destination,p in Port} Pijkp[i,j,k,p]*Wij[i,j]) + (sum{i in Origin, j in Destination,  m in PCTL, p in Port} Pijkmp[i,j,k,m,p]*Wij[i,j])  <= CAPk[k]*Yk[k];
    subject to flow2 {m in PCTL}:#garantizan que el flujo de carga que maneja cada terminal es menor o igual a la capacidad instalada
        (sum{i in Origin, j in Destination,p in Port} Pijmp[i,j,m,p]*Wij[i,j]) + (sum{i in Origin, j in Destination,  k in CTL, p in Port} Pijkmp[i,j,k,m,p]*Wij[i,j])  <= CAPm[m]*Ym[m];
 
            
        """)

Licensed to AMPL Community Edition License for <santiagotorreglosadiaz@gmail.com>.


In [4]:
# Leemos los conjuntos
import pandas as pd
import numpy as np
import sys
import os


Origin = pd.DataFrame([
    ('manizales'),
    ('medellin'),
    ('bucaramanga'),
    ('bogota'),
    ('ibague'),
    ('neival'),
    ('soacha'),
    ('pereira'),
    ],
    columns=["Origin"],)#.set_index("Origin")

Destination = pd.DataFrame([
    ('españa'),
    ('alemania'),
    ('brazil'),
    ('panama'),
    ('mexico'),
    ('estados_unidos'),
    ],
    columns=["Destination"],)#.set_index("Destination")


CTL = pd.DataFrame([
    ('ctl_bogota'),
    ('ctl_pereira'),
    ('ctl_manizales'),
    ('ctl_soacha'),
    ('ctl_neiva'),
    ('ctl_ibague'),
    ('ctl_cienaga'),
    ('ctl_la_dorada'),
    ('ctl_puerto_berrio'),
    ],
    columns=["CTL"],)#.set_index("CTL")

PCTL = pd.DataFrame([
    ('ctl_soeldad'),
    ('ctl_santa_marta'),
    ('ctl_cartagena'),
    ],
    columns=["PCTL"],)#.set_index("PCTL")

Port = pd.DataFrame([
    ('puerto_de_cartagena'),
    ('puerto_de_barranquilla'),
    ('puerto_de_santa_marta'),
    ],
    columns=["Port"],)#.set_index("Port")

CAPk = pd.read_excel("model_data.xlsx",sheet_name="CAPk").set_index("CTL")
CAPm = pd.read_excel("model_data.xlsx",sheet_name="CAPm").set_index("PCTL")

Fk = pd.read_excel("model_data.xlsx",sheet_name="Fk").set_index("CTL")
Fm = pd.read_excel("model_data.xlsx",sheet_name="Fm").set_index("PCTL")

Wij=pd.read_excel("model_data.xlsx",sheet_name="Wij").set_index(['Origin', 'Destination'])

costs=pd.read_excel("model_data.xlsx",sheet_name="cost")
costs.set_index(['code_inicio', 'code_fin'], inplace=True)

costs


code_inicio_type  \
code_inicio                     code_fin                                           
puerto_de_cartagena             panama                                         p   
                                mexico                                         p   
                                estados_unidos                                 p   
                                españa                                         p   
                                alemania                                       p   
...                                                                          ...   
estacion_grecia                 estacion_barrancabermeja                       f   
estacion_barrancabermeja        estacion_chiriguana                            f   
estacion_chiriguana             estacion_fererocarril_fundacion                f   
estacion_fererocarril_fundacion estacion_ferrocarril_de_cienaga                f   
estacion_ferrocarril_de_cienaga estacion_santa_marta                           f   

                                                                code_fin_type  \
code_inicio                     code_fin                                        
puerto_de_cartagena             panama                                      j   
                                mexico                                      j   
                                estados_unidos                              j   
                                españa                                      j   
                                alemania                                    j   
...                                                                       ...   
estacion_grecia                 estacion_barrancabermeja                    f   
estacion_barrancabermeja        estacion_chiriguana                         f   
estacion_chiriguana             estacion_fererocarril_fundacion             f   
estacion_fererocarril_fundacion estacion_ferrocarril_de_cienaga             f   
estacion_ferrocarril_de_cienaga estacion_santa_marta                        f   

                                                                 transport_cost  \
code_inicio                     code_fin                                          
puerto_de_cartagena             panama                              5641.600000   
                                mexico                             27714.933333   
                                estados_unidos                     20353.333333   
                                españa                             86435.733333   
                                alemania                          100654.400000   
...                                                                         ...   
estacion_grecia                 estacion_barrancabermeja             111.111111   
estacion_barrancabermeja        estacion_chiriguana                  290.666667   
estacion_chiriguana             estacion_fererocarril_fundacion      132.444444   
estacion_fererocarril_fundacion estacion_ferrocarril_de_cienaga      102.222222   
estacion_ferrocarril_de_cienaga estacion_santa_marta                  30.222222   

                                                                 externality  
code_inicio                     code_fin                                      
puerto_de_cartagena             panama                            2840.16348  
                                mexico                           13952.59173  
                                estados_unidos                   10246.52475  
                                españa                           43514.53722  
                                alemania                         50672.67282  
...                                                                      ...  
estacion_grecia                 estacion_barrancabermeja           725.00000  
estacion_barrancabermeja        estacion_chiriguana               1896.60000  
estacion_chiriguana             estacion_ferero

In [3]:
# Origin 
# Destination
# CTL 
# PCTL 
# Port
# CAPk 
# CAPm
# Fk
# Fm
# Wij
# costs
valor = costs.loc[('puerto_de_cartagena', 'mexico'), 'transport_cost']
valor
#Cijp
#Cijkp
#Cijmp
#Cijkmp

#Eijp
#Eijkp
#Eijmp
#Eijkmp
#Wij 


/home/storreglosa/anaconda3/envs/multimodal_logistics_network_colombia/lib/python3.7/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


code_inicio          code_fin
puerto_de_cartagena  mexico      27714.933333
Name: transport_cost, dtype: float64

In [14]:
O=list(Origin['Origin'])
D=list(Destination['Destination'])
ks=list(CTL['CTL'])
ms=list(PCTL['PCTL'])
puer=list(Port['Port'])


#Cijp
Cijp=[]
for i in O:
    for j in D:
        for p in puer:
            try:
                valor=costs.loc[(i, p), 'transport_cost'] + costs.loc[(i, j), 'transport_cost']
                diccionario = {"i":i,"j":j,"p":p,"Cijp":valor}
                Cijp.append(diccionario)
            except:
                valor = np.inf
                diccionario = {"i":i,"j":j,"p":p,"Cijp":valor}
                Cijp.append(diccionario)
Cijp =pd.DataFrame(Cijp)
#Cijkp
Cijkp=[]
for i in O:
    for j in D:
        for k in ks:
            for p in puer:
                try:
                    valor=costs.loc[(i,k), 'transport_cost'] + costs.loc[(k,p), 'transport_cost'] + costs.loc[(p,j), 'transport_cost']
                    diccionario = {"i":i,"j":j,"k":k,"p":p,"Cijp":valor}
                    Cijkp.append(diccionario)
                    #print(valor)
                except:
                    valor = np.inf
                    #print(valor)
                    diccionario = {"i":i,"j":j,"k":k,"p":p,"Cijkp":valor}
                    Cijkp.append(diccionario)
Cijkp =pd.DataFrame(Cijkp)

#Cijmp
Cijmp=[]
for i in O:
    for j in D:
        for m in ms:
            for p in puer:
                try:
                    valor=costs.loc[(i,m), 'transport_cost'] + costs.loc[(m,p), 'transport_cost'] + costs.loc[(p,j), 'transport_cost']
                    diccionario = {"i":i,"j":j,"m":m,"p":p,"Cijmp":valor}
                    Cijmp.append(diccionario)
                    #print(valor)
                except:
                    valor = np.inf
                    #print(valor)
                    diccionario = {"i":i,"j":j,"m":m,"p":p,"Cijmp":valor}
                    Cijmp.append(diccionario)
Cijmp =pd.DataFrame(Cijmp)

#a=pd.Series(Cijmp["Cijmp"].unique())
#Cijkmp
#Eijp
#Eijkp
#Eijmp
#Eijkmp
#Wij
